## Handling Imbalaced Datasets with Machine Learning

In [1]:
!pip install imbalanced-learn

In [3]:
import pandas as pd
df = pd.read_csv("creditcard.csv")

In [4]:
df.shape

(284807, 31)

In [5]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [12]:
df["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [15]:
X = df.drop(["Class"],axis=1)
y = df["Class"]

### 1.Crass Validation Like KFOLD and Hyperparameter Tuning

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import numpy as np

In [22]:
10.0**np.arange(-2,3)

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [23]:
log_class_obj = LogisticRegression()
grid = {"C":10.0**np.arange(-2,3),"penalty":["l1","l2"]}
cv = KFold(n_splits=5, shuffle=False, random_state= None)

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size= 0.7)

In [27]:
clf= GridSearchCV(log_class_obj,grid,cv=cv,n_jobs=-1,scoring="f1_macro")
clf.fit(X_train,y_train)

C:\Users\mwolf\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [28]:
clf.best_params_

{'C': 0.01, 'penalty': 'l2'}

In [29]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85285    22]
 [   54    82]]
0.9991105181231933
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.79      0.60      0.68       136

    accuracy                           1.00     85443
   macro avg       0.89      0.80      0.84     85443
weighted avg       1.00      1.00      1.00     85443



In [38]:
from sklearn.ensemble import RandomForestClassifier
classifier_obj = RandomForestClassifier()
classifier_obj.fit(X_train,y_train)

RandomForestClassifier()

In [39]:
y_pred = classifier_obj.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85301     6]
 [   24   112]]
0.9996488887328394
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.95      0.82      0.88       136

    accuracy                           1.00     85443
   macro avg       0.97      0.91      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [41]:
class_weight = dict({0:1,1:100})

In [42]:
from sklearn.ensemble import RandomForestClassifier
classifier_obj = RandomForestClassifier(class_weight=class_weight)
classifier_obj.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [43]:
y_pred = classifier_obj.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85301     6]
 [   25   111]]
0.9996371850239341
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.95      0.82      0.88       136

    accuracy                           1.00     85443
   macro avg       0.97      0.91      0.94     85443
weighted avg       1.00      1.00      1.00     85443



### 2.Under Sampling

In [44]:
y.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [53]:
from collections import Counter
from imblearn.under_sampling import NearMiss
ns_obj = NearMiss(0.8)
X_train_ns,y_train_ns = ns_obj.fit_sample(X_train,y_train)
print("The number of classes before fit{}".format(Counter(y_train)))
print("The number of classes after fit{}".format(Counter(y_train_ns)))

C:\Users\mwolf\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fitCounter({0: 199008, 1: 356})
The number of classes after fitCounter({0: 445, 1: 356})


In [54]:
from sklearn.ensemble import RandomForestClassifier
classifier_obj = RandomForestClassifier()
classifier_obj.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [56]:
y_pred = classifier_obj.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[73757 11550]
 [    7   129]]
0.8647402361808457
              precision    recall  f1-score   support

           0       1.00      0.86      0.93     85307
           1       0.01      0.95      0.02       136

    accuracy                           0.86     85443
   macro avg       0.51      0.91      0.47     85443
weighted avg       1.00      0.86      0.93     85443



### 3. Over Sampling

In [57]:
from imblearn.over_sampling import RandomOverSampler

In [62]:
os_obj = RandomOverSampler(0.8)
X_train_ns,y_train_ns = os_obj.fit_sample(X_train,y_train)
print("The number of classes before fit{}".format(Counter(y_train)))
print("The number of classes after fit{}".format(Counter(y_train_ns)))

C:\Users\mwolf\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fitCounter({0: 199008, 1: 356})
The number of classes after fitCounter({0: 199008, 1: 159206})


In [63]:
from sklearn.ensemble import RandomForestClassifier
classifier_obj = RandomForestClassifier()
classifier_obj.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [64]:
y_pred = classifier_obj.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85301     6]
 [   26   110]]
0.9996254813150287
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.95      0.81      0.87       136

    accuracy                           1.00     85443
   macro avg       0.97      0.90      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [65]:
0.8*199008

159206.40000000002

### 4.SMOTETomek

In [68]:
from imblearn.combine import SMOTETomek

In [74]:
os_obj = SMOTETomek(0.5)
X_train_ns,y_train_ns = os_obj.fit_sample(X_train,y_train)
print("The number of classes before fit{}".format(Counter(y_train)))
print("The number of classes after fit{}".format(Counter(y_train_ns)))

C:\Users\mwolf\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


The number of classes before fitCounter({0: 199008, 1: 356})
The number of classes after fitCounter({0: 198243, 1: 98739})


In [75]:
from sklearn.ensemble import RandomForestClassifier
classifier_obj = RandomForestClassifier()
classifier_obj.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [76]:
y_pred = classifier_obj.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85296    11]
 [   19   117]]
0.9996488887328394
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.91      0.86      0.89       136

    accuracy                           1.00     85443
   macro avg       0.96      0.93      0.94     85443
weighted avg       1.00      1.00      1.00     85443



### 5.Ensemble Techniques

In [77]:
from imblearn.ensemble import EasyEnsembleClassifier

In [78]:
easy_obj = EasyEnsembleClassifier()
easy_obj.fit(X_train,y_train)

EasyEnsembleClassifier()

In [79]:
y_pred = easy_obj.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[82170  3137]
 [    7   129]]
0.963203539201573
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     85307
           1       0.04      0.95      0.08       136

    accuracy                           0.96     85443
   macro avg       0.52      0.96      0.53     85443
weighted avg       1.00      0.96      0.98     85443

